In [ ]:
!pip install transformers torch datasets evaluate

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,  # Changed from AutoConfing to AutoConfig
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

import evaluate
import torch
import numpy as np

##Import Model and Define Labels

In [ ]:
model_checkpoint = 'distilbert-base-uncased'

#Define as Labels

id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

In [ ]:
#Gera o modelo de classificação com checkpoint
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset = load_dataset("shawhin/imdb-truncated")

dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

##Tokenização

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)


In [ ]:

def tokenize_function(examples):
	text = examples["text"]

	tokenizer.truncation_side = "left"
	tokenized_inputs = tokenizer(
	text,
	return_tensors="np",
	truncation=True,
	max_length=512
)
	return tokenized_inputs


if tokenizer.pad_token is None:
	tokenizer.add_special_tokens({'pad_token': '[PAD]'})
	model.resize_token_embeddings(len(tokenizer))

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

##Metrics

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
	predictions, labels = p
	predictions = np.argmax(predictions, axis=1)

	return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

##Evaluate model untrained

In [ ]:
text_list = ["It was good","Not as fan","don't recommed.","Better than the first one", "This is not worth watching even once","This one is a pass."]

print("Untrained model predictions:")
print("--------------------------")

for text in text_list:
	inputs = tokenizer.encode(text, return_tensors="pt")

	logits = model(inputs).logits

	predictions = torch.argmax(logits)

	print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
--------------------------
It was good - Negative
Not as fan - Negative
don't recommed. - Negative
Better than the first one - Negative
This is not worth watching even once - Positive
This one is a pass. - Positive


In [ ]:
peft_config = LoraConfig(task_type="SEQ_CLS",
			r=4,
			lora_alpha=32,
			lora_dropout=0.01,
			target_modules = ['q_lin'])

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [ ]:
lr = 1e-3
batch_size = 4
num_epochs = 10

training_args = TrainingArguments(
output_dir = model_checkpoint + "-lora-text-classification",
learning_rate=lr,
per_device_train_batch_size=batch_size,
per_device_eval_batch_size=batch_size,
num_train_epochs=num_epochs,
weight_decay=0.01,
evaluation_strategy="epoch",
save_strategy="epoch",
load_best_model_at_end=True)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
	model=model,
	args=training_args,
	train_dataset=tokenized_dataset["train"],
	eval_dataset=tokenized_dataset["validation"],
	tokenizer=tokenizer,
	data_collator=data_collator,
	compute_metrics=compute_metrics)

trainer.train()

<ipython-input-39-92426934d095>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.478298,{'accuracy': 0.851}
2,0.442600,0.433509,{'accuracy': 0.874}
3,0.442600,0.645478,{'accuracy': 0.866}


Trainer is attempting to log a value of "{'accuracy': 0.851}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.874}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.866}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.478298,{'accuracy': 0.851}
2,0.442600,0.433509,{'accuracy': 0.874}
3,0.442600,0.645478,{'accuracy': 0.866}
4,0.209200,0.702205,{'accuracy': 0.879}
5,0.209200,0.840618,{'accuracy': 0.885}


Trainer is attempting to log a value of "{'accuracy': 0.879}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.885}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [ ]:
model.to('mps')

print("Trained model predictions:")
print("-------------------------")

for text in text_list:
	inputs = tokenizer.encode(text, return_tensors="pt").to("mps")

	logits = model(inputs).logits
	predictions = torch.max(logits,1).indices

	print(text + " - " + id2label[predictions.tolist()[0]])